# Import Modules

In [1]:
import os
print(os.getcwd())
import sys

import json

import pandas as pd

# #########################################################
sys.path.insert(0, os.path.join(os.environ["PROJ_irox"], "data"))
from proj_data_irox import calc_dH

sys.path.insert(0, os.path.join(os.environ["PROJ_irox"], "workflow/ml_modelling"))
from ml_methods import get_ml_dataframes

/mnt/f/Dropbox/01_norskov/00_git_repos/PROJ_IrOx_Active_Learning_OER/workflow/07_bulk_pourbaix/03_data_table


In [2]:
# #######################################################################
data_path = os.path.join(
    os.environ["PROJ_irox"],
    "workflow/energy_treatment_deriv/calc_references",
    "out_data/data.json")
with open(data_path, "r") as fle:
    data = json.load(fle)
# #######################################################################

h_ref = data['h_ref']
o_ref = data['o_ref']
ir_ref = data['ir_ref']

TdS_IrO2 = data['TdS_iro2']
TdS_IrO3 = data['TdS_iro3']
TdS_IrHO3 = data['TdS_irho3']

# Script Inputs

In [3]:
# TdS_IrO2 = -0.562803124838058
# TdS_IrO3 = -0.789056096134837

mu_H2O = -2.4583

# #########################################################
dH_r_iro2 = -2.515126703632689
dH_b_iro3 = 4 * -0.596831474814422

# #########################################################
dG_ir_aq = -2.038081775

In [4]:
all_systems = [
    "ir",
    "r-iro2",
    "a-iro3",
    "r-iro3",
    "b-iro3",
    "ir_aq",
    "a-irho3",
    ]
    
TdS_dict = {
    "AB2": TdS_IrO2,
    "AB3": TdS_IrO3,
    "IrHO3": TdS_IrHO3,
    }

# Read Data

## Read DFT data for a-IrO3 and R-IrO3

In [5]:
DF_dict = get_ml_dataframes()

df_dft = DF_dict.get("df_dft_final_final")

df_i = df_dft[
    (df_dft.stoich == "AB3")
    ].sort_values("dH")

alpha_iro3_row = df_i.iloc[0]
rutile_iro3_row = df_i.loc["b5cgvsb16w"]

# Put together Enthalpy dict

In [6]:
dH_dict = {
    "ir": 0.,
    "r-iro2": dH_r_iro2,
    "a-iro3": 4 * alpha_iro3_row.dH,
    "r-iro3": 4 * rutile_iro3_row.dH,
    "b-iro3": dH_b_iro3,
    "a-irho3": (5 * calc_dH(-6.151177564999999, stoich="IrHO3")),
    # "ir_aq": None,
    }

# (5 * calc_dH(-6.151177564999999, stoich="IrHO3")) - TdS_irho3
# dH_dict

# TEMP

In [7]:
dG_dict = dict()

for sys_i, dH_i in dH_dict.items():

    if "iro2" in sys_i:
        stoich_i = "AB2"
    elif "iro3" in sys_i:
        stoich_i = "AB3"
    elif "irho3" in sys_i:
        stoich_i = "IrHO3"
    else:
        continue

    # Getting Entropy correction
    TdS_i = TdS_dict[stoich_i]


    dG = dH_i - TdS_i
    dG_dict[sys_i] = dG


# TdS_IrO2 = -0.562803124838058
# TdS_IrO3 = -0.789056096134837

dG_dict["ir_aq"] = dG_ir_aq
dG_dict["ir"] = 0.

In [8]:
data_dict_list = []
for sys_i in all_systems:
    # print(sys_i)

    data_dict_i = dict()
    data_dict_i["system"] = sys_i
    
    
    data_dict_i["dH"] = dH_dict.get(sys_i, None)
    data_dict_i["dG"] = dG_dict.get(sys_i, None)
    
    data_dict_list.append(data_dict_i)

df = pd.DataFrame(data_dict_list)
df

,system,dH,dG
0,ir,0.000000,0.000000
1,r-iro2,-2.515127,-1.952324
2,a-iro3,-2.602809,-1.813753
3,r-iro3,-2.551268,-1.762212
4,b-iro3,-2.387326,-1.598270
5,ir_aq,NaN,-2.038082
6,a-irho3,-4.273061,-3.210571


In [9]:
num_dec = 3

df = df.round({
    "dH": num_dec,
    "dG": num_dec,
    })

In [10]:
units_dict = {
    "system": "",
    "dH": "(eV/f.u.)",
    "dG": "(eV/f.u.)",
    }

units_list = []
for col_i in df.columns.values:
    unit_i = units_dict[col_i]
    units_list.append(unit_i)

tuples = list(zip(*[
    df.columns.values,
    units_list,
    ]))

index = pd.MultiIndex.from_tuples(
    tuples,
    # names=["Header", "Units"],
    )
df.columns = index

In [11]:
column_rename_dict = {
    "dH": "$\Delta H_{f}$",
    "dG": "$\Delta G_{f}$",
    }

df = df.rename(columns=column_rename_dict)


df = df.replace(
    to_replace="ir",
    value="Ir(s)")

df = df.replace(
    to_replace="r-iro2",
    value="$R$-$IrO_{2}(s)$")

df = df.replace(
    to_replace="a-iro3",
    value="$\\alpha$-$IrO_{3}(s)$")
df = df.replace(
    to_replace="r-iro3",
    value="$R$-$IrO_{3}(s)$")
df = df.replace(
    to_replace="b-iro3",
    value="$\\beta$-$IrO_{3}(s)$")
df = df.replace(
    to_replace="ir_aq",
    value="$IrO_{4}^{-}(aq)$")
df = df.replace(
    to_replace="a-irho3",
    value="$\\alpha$-$IrHO_{3}(s)$")

In [12]:
alignment_str = "lcc"

# Write latex table to file

In [13]:
shared_props = dict(
    # buf="oer_table.tex",
    columns=None,
    col_space=None,

    # #####################################################
    header=True,
    # #####################################################
    index=False,

    # #####################################################
    na_rep='-',
    formatters=None,

    # #####################################################
    # float_format="{:0.2f}",
    sparsify=None,
    index_names=True,
    bold_rows=False,
    
    # #####################################################
    column_format=alignment_str,
    
    longtable=None,
    
    # #####################################################
    escape=False,
    encoding=None,
    decimal='.',
    multicolumn=None,
    multicolumn_format=None,
    multirow=None,
    )

In [14]:
df.to_latex(
    buf="bulk_pourb_table.tex",
    **shared_props)

path_i = os.path.join(
    os.environ["PROJ_irox_paper"],
    "04_data_tables/bulk_pourb_energy",
    "bulk_pourb_table.tex")

df.to_latex(
    buf=path_i,
    **shared_props)

In [15]:
df

,system,$\Delta H_{f}$,$\Delta G_{f}$
,,(eV/f.u.),(eV/f.u.)
0,Ir(s),0.000,0.000
1,$R$-$IrO_{2}(s)$,-2.515,-1.952
2,$\alpha$-$IrO_{3}(s)$,-2.603,-1.814
3,$R$-$IrO_{3}(s)$,-2.551,-1.762
4,$\beta$-$IrO_{3}(s)$,-2.387,-1.598
5,$IrO_{4}^{-}(aq)$,NaN,-2.038
6,$\alpha$-$IrHO_{3}(s)$,-4.273,-3.211
